In [18]:
import numpy as np
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR100      
from torch.utils.data import DataLoader
import timm
import detectors
from src.saps import split_data_set, saps_scores, saps_classification, eval_aps
from src.temperature_scaling import ModelWithTemperature

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=1.0).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# error rate
alpha = 0.1
lambda_ = 0.17
num_runs = 10

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
SAPS Classification, Start!

Running experiment 1/10...
t_cal = 1.0456591725349427
Total set size: 9149
Total coverage sets: 4539
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.8298
Average Coverage Rate in runs 1: 0.9078

Running experiment 2/10...
t_cal = 1.0279643893241885
Total set size: 8725
Total coverage sets: 4497
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.745
Average Coverage Rate in runs 2: 0.8994

Running experiment 3/10...
t_cal = 1.031078600883484
Total set size: 8806
Total coverage sets: 4496
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.7612
Average Coverage Rate in runs 3: 0.8992

Running experiment 4/10...
t_cal = 1.0141916394233703
Total set size: 8270
Total coverage sets: 4464
Total samples amount: 5000
Average Prediction Set Size After APS in runs 4: 1.6

# Result

- Final Average **Prediction Set Size： 1.74 / 100**
- Final Average **Coverage: 89.86% ($\alpha$=0.1)**